In [5]:
import requests
from bs4 import BeautifulSoup

#### request 보내기

In [159]:
soup.find("nav")

<nav id="navi">
<div class="wrap_pc_nav">
<div class="pc_nav">
<!-- PC 네비게이션 -->
</div>
<div class="nav_d2" tabindex="-1">
<!-- PC 네이게이션 더보기 -->
</div>
<div class="progress_container">
<div class="progress_bar" id="progressBar"></div>
</div>
</div>
<!-- 모바일 m_navi -->
</nav>

In [164]:
soup.select('#rollHashtag')

[<ul id="rollHashtag">
 <li><a href="#">날씨</a></li>
 <li><a href="#">코로나19</a></li>
 <li><a href="#">사회적 거리두기</a></li>
 <li><a href="#">국회</a></li>
 <li><a href="#">페이스북</a></li>
 <li><a href="#">안산 유치원</a></li>
 <li><a href="#">이재용</a></li>
 <li><a href="#">인천국제공항</a></li>
 <li><a href="#">이스타항공</a></li>
 <li><a href="#">이상직</a></li>
 </ul>]

b'\xef\xbb\xbf\r\n\r\n\r\n\r\n\r\n<!DOCTYPE html>\r\n<html lang="ko">\r\n\r\n<head>\r\n    <meta charset="utf-8">\r\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n     <meta  http-equiv="Expires" content="-1">\r\n     <meta  http-equiv="Pragma" content="no-cache">\r\n     <meta  http-equiv="Cache-Control" content="no-cache, no-store, must-revalidate">  \r\n    <meta name="viewport" content="width=device-width,initial-scale=1.0,minimum-scale=1.0,maximum-scale=1.0,user-scalable=no">\r\n    <meta name="title" content="MBC NEWS">\r\n    <meta name="url" content="https://imnews.imbc.com/news/2020/politics/5862149_32621.html"> \r\n    <meta name="description" content="">\r\n    <meta name="keywords" content=""> \r\n    <meta property="og:url" content="https://imnews.imbc.com/news/2020/politics/5862149_32621.html"> \r\n    <meta property="og:title" content="\xec\xa0\x95\xec\xb9\x98">\r\n    <meta property="og:description" content="">  \r\n    <meta property="og:site_name" conte

In [163]:
url = 'https://imnews.imbc.com/pc_main.html'

resp = requests.request('GET',url)
soup = BeautifulSoup(resp.content)
soup

<!DOCTYPE html>
<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="-1" http-equiv="Expires"/>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="no-cache" http-equiv="Cache-Control"/>
<meta content="width=1300" name="viewport"/>
<meta content="MBC NEWS" name="title"/>
<meta content="http://imnews.imbc.com/" name="url"/>
<meta content="기본과 원칙에 충실한 뉴스" name="description"/>
<meta content="http://imnews.imbc.com/" property="og:url"/>
<meta content="MBC NEWS" property="og:title"/>
<meta content="기본과 원칙에 충실한 뉴스" property="og:description"/>
<meta content="MBC NEWS" property="og:site_name"/>
<meta content="https://image.imnews.imbc.com/page/include/images/meta_image.png" property="og:image"/>
<meta content="https://www.facebook.com/MBCnews" property="article:publisher"/>
<meta content="ko_KR" property="og:locale"/>
<meta content="692275414241491" property="fb:app_id"/>
<meta content="@mbcnews" name="twitter:site"/>
<

#### 상품 리스트 가져오기

In [145]:
 
ul_tag = soup.select("div.wrap_pc_nav wrap_pc_nav.d1")
print(ul_tag)


[]


## 정치

In [156]:
politics_list=[]

url = 'https://imnews.imbc.com/news/2020/politics/'
resp = requests.request('GET',url)
soup = BeautifulSoup(resp.text)

sec_tag = soup.find("section")
span_tag = sec_tag.find_all("span",class_="txt_w")


for i in span_tag:
    title=i.find("span", class_="tit")
    print(title)

<span class="tit ellipsis">이낙연 "당에 대한 민심 냉랭해지고 있다"</span>
<span class="tit ellipsis2">靑 "강제징용 기업 자산압류는 법원 결정…입장 표명 부적절"</span>
<span class="tit ellipsis2">외교부 "성추행 의혹 외교관 즉각 귀임 발령"</span>
<span class="tit ellipsis2">법사위 여야 설전…"자질 없다" vs "가짜 뉴스"</span>
<span class="tit ellipsis2">문 대통령 "인명피해 최소화…구조대 안전 유의"</span>
<span class="tit ellipsis2">'해외 입국' 주한미군 7명 코로나19 확진…누적 133명</span>
<span class="tit ellipsis2">문 대통령 "인명피해 최소화를 최우선으로…구조 중 희생도 없어야"</span>
<span class="tit ellipsis2">정 총리 "방역과 일상 조화 찾는 노력…산발적 해외유입 차단 노력"</span>
<span class="tit ellipsis2">당정, 내일 주택공급 방안 협의…재건축 용적률 상향 등 검토 </span>
<span class="tit ellipsis2">조수진 "법사위원 그만둬야" 김종민 "전형적인 말꼬리"</span>
<span class="tit ellipsis2">광주 찾은 박주민 "헌법재판소 광주 이전 제안"</span>
<span class="tit ellipsis2">박병석 의장, 여야 중진들과 첫 오찬 "타협의 국회 만들자"</span>
<span class="tit ellipsis2">통합당, 이스타 의혹 TF 출범…"이상직 의원이 직접 밝혀야"</span>
<span class="tit ellipsis2">외교부, 뉴질랜드 성추행 의혹 외교관 귀임 조치</span>
<span class="tit ellipsis2">문 대통령, 폭우 피해에 여름 휴가 취소</span>
